# Simplified Quantum Terrain Generation

A blank circuit, or one with far fewer gates than there are qubits, will lead to a very spiky result: some outputs will be likely, but most will be very rare. At the other extreme, a circuit with a Hadamard on all qubits creates a very smooth result. But by creating quantum interference effects, we can get more interesting distributions than these simple cases.

To make this more obvious, we will make the output of a circuit into an island. A smooth or spiky distribution will give a smooth or spiky island. A distribution that shows interesting features of quantum interference, however, will have a more interesting texture.

---

The following cell is a big hacky function that takes a statevector, turns it into some terrain, and then saves that terrain as a csv.

In [1]:
import numpy as np
    
def ket2csv(statevector,size=[50,50,20],terrain=[2/16,4/16,7/16,12/16,15/16]):

    # determine how many qubits were in the circuit
    num = int(np.log(len(statevector))/np.log(2))

    # determine side lengths for a grid with 2**n points, and the number of bits needed to represent their coords
    n = {}
    n['y'] = int(num/2)
    n['x'] = int(num/2)
    L = {}
    L['y'] = 2**n['y']
    L['x'] = 2**n['x']

    # assign a bit string to each point on the grid, so that Hamming distance <= Manhattan distance
    # the string for a point (x,y) will be coords['x'][x]+coords['y'][y]
    coords = {}
    for axis in ['x','y']:
        coords[axis] = ['0','1']
        for j in range(n[axis]-1):
            coords[axis] = coords[axis] + coords[axis][::-1]
            for j in range(int(len(coords[axis])/2)):
                coords[axis][j] += '0'
            for j in range(int(len(coords[axis])/2),int(len(coords[axis]))):
                coords[axis][j] += '1'

    # create a heightmap for which each height is the counts value for the corresponding string
    tartan = {}
    for x in range(L['x']):
        for y in range(L['y']):
            try:
                amp = statevector[int(coords['x'][x]+coords['y'][y],2)]
            except:
                amp = 0
            tartan[x,y] = np.real(amp*np.conj(amp))

    def normalizeZ(Z,min0=True):
        maxZ = 0
        minZ = 100
        for coord in Z:
            maxZ = max(maxZ,Z[coord])
            minZ = min(minZ,Z[coord])
        if not min0:
            minZ = 0
        for coord in Z:
            if maxZ>minZ:
                Z[coord] = (Z[coord]-minZ)/(maxZ-minZ)
            else:
                Z[coord] = 0
        return Z

    tartan = normalizeZ(tartan,min0=False)
    
    # initialize heightmap to be a smooth hill
    Z = {}
    for x in range(size[0]):
        for y in range(size[1]):
            Z[x,y] = np.cos(np.sqrt( (x-size[0]/2)**2 + (y-size[1]/2)**2 + 0.1 )/(min(size[0],size[1])/5))

    # fill it with tartan
    for x0 in range(0,size[0],int(L['x'])):
        for y0 in range(0,size[1],int(L['y'])):
            for (x,y) in tartan:
                xx = x+x0
                yy = y+y0
                if (xx,yy) in Z:
                    Z[xx,yy] += tartan[x,y]

    # normalize
    Z = normalizeZ(Z)
       
    # make and save the csv, which assigns blocks to coordinates
    
    def make_blocks(Z,terrain=terrain,height=int(size[2]/2),depth=int(size[2]/2)):
        # make a dictionary that determines which material exists at each 3D position
        # also returns mins and maxs of all three coordinates in `mins` and `maxs`
        def addBlocks( blocks, x1,h1,y1, x2,h2,y2, block ):
            # add a blocks of a given type for a given range of coordinates
            for x in range(x1,x2+1):
                for y in range(y1,y2+1):
                    for h in range(h1,h2+1):
                        blocks[x,h,y] = block

        def addTreeBlocks( blocks, x,h,y, rnd ):
            #Makes a tree, rooted at the specified position'''
            for j in range(1,6):
                blocks[x,h+j,y] = 'tree'
            for xx in range(x-3,x+4):
                for yy in range(y-3,y+4):
                    for hh in range(h+5,h+11):
                        d = (xx-x)**2+(yy-y)**2+(hh-h-6)**2 + 0.1
                        if d<8:
                            blocks[xx,hh,yy] = 'leaves'
            xx = choose([x-1,x+1],rnd)
            yy = choose([y-1,y+1],rnd)
            blocks[xx,h+5,yy] = 'tree'
            blocks[xx,h+4,yy] = 'torch'

        def choose( options, rnd ):
            return options [ int(round(rnd*( len(options)-1 ))) ]

        sea_level = int( depth+terrain[0]*height+1 )

        spawn = [int(size[0]/2),size[2],int(size[1]/2)]

        blocks = {}
        (Xmin,Hmin,Ymin) = (0,0,0)
        (Xmax,Hmax,Ymax) = (0,0,0)
        for (X,Y) in Z:

            Hfloat = depth + Z[X,Y]*height
            H = int( Hfloat ) # height for a block
            rnd = Hfloat-H # value from 0 to 1 that we can use for randomness

            Xmin = min(Xmin,X); Ymin = min(Ymin,Y); Hmin = min(Hmin,H)
            Xmax = max(Xmax,X); Ymax = max(Ymax,Y); Hmax = max(Hmax,H)

            # First we make a cavern, which is most spacious under hills

            Hm = int( (1-Z[X,Y])*depth/2 ) # height for stalagtites
            Ht = int( depth - (1-Z[X,Y])*depth/2 ) #height at which stalagmites begin

            if Z[X,Y]<terrain[0]:
                minerals = ['diamondblock','goldblock'] # most precious minerals in hard to reach places
            else:
                minerals = ['stone','stone','stone_with_coal','stone_with_iron','stone_with_copper','stone_with_tin','stone_with_gold','stone_with_diamond']
            stone_m = choose(minerals,rnd)
            stone_t = choose(minerals,1-rnd)

            if (1-Z[X,Y])<terrain[0]: # the very bottom of the cavern has lava
                blocks[X,0,Y] = stone_m
                blocks[X,1,Y] = 'lava_source'
            else: # otherwise a mineral
                addBlocks( blocks, X,0,Y, X,Hm,Y, stone_m )

            if Z[X,Y]<terrain[4]: # the roof is always a mineral
                addBlocks( blocks, X,Ht,Y, X,depth,Y, stone_t )

            if rnd<0.005 and Z[X,Y]>terrain[0] and Z[X,Y]<terrain[4]:
                blocks[X,Ht-1,Y] = 'torch'

            if Z[X,Y]<terrain[0]: # sand at H and then water up to sea level
                addBlocks( blocks, X,depth,Y, X,H,Y, 'sand' )
                addBlocks( blocks, X,H+1,Y, X,sea_level,Y, 'water_source' )
            elif Z[X,Y]<terrain[1]: # sand
                addBlocks( blocks, X,depth+1,Y, X,H-1,Y, 'stone' )
                blocks[X,H,Y] = 'sand'
                blocks[X,H+1,Y] = 'sand'
            elif Z[X,Y]<terrain[2]: # grass with trees
                addBlocks( blocks, X,depth+1,Y, X,H-1,Y, 'stone' )
                blocks[X,H,Y] = 'dirt_with_grass'
                blocks[X,H+1,Y] = 'dirt_with_grass'
                if rnd<0.025:
                    addTreeBlocks( blocks, X,H,Y, rnd )
                else:
                    blocks[X,H+2,Y] = choose(['fern_1','marram_grass_1','marram_grass_2','marram_grass_3'],rnd)
            elif Z[X,Y]<terrain[3]: # grass with ferns
                addBlocks( blocks, X,depth+1,Y, X,H-1,Y, 'stone' )
                blocks[X,H,Y] = 'dirt_with_grass'
                blocks[X,H+1,Y] = 'dirt_with_grass'
                blocks[X,H+2,Y] = choose(['fern_1','fern_2','fern_3','marram_grass_1'],rnd)
            elif Z[X,Y]<terrain[4]: # mixture of grass and stone
                addBlocks( blocks, X,depth+1,Y, X,H-1,Y, 'stone' )
                if rnd<1/3:
                    blocks[X,H,Y] = 'dirt_with_grass'
                    blocks[X,H+1,Y] = 'dirt_with_grass'
                else:
                    blocks[X,H,Y] = 'stone'
                    blocks[X,H+1,Y] = 'stone'
            elif H==(depth + terrain[4]*height): # just stone, with a random bit of additional height
                H += int(height*rnd/10)
                addBlocks( blocks, X,depth+1,Y, X,H+1,Y, 'stone' )

            mins = (Xmin,Hmin,Ymin)
            maxs = (Xmax,Hmax,Ymax)

        return blocks, spawn, mins, maxs
    
    def save_blocks(blocks,spawn,mins,maxs,filename='blocks.csv'):
        # saves a dictionary of the form created by the above function as a csv file
        with open(filename, 'w') as file:
            file.write( str(mins[0])+','+str(mins[1])+','+str(mins[2])+',min,\n' )
            file.write( str(maxs[0])+','+str(maxs[1])+','+str(maxs[2])+',max,\n' )
            file.write( str(spawn[0])+','+str(spawn[1])+','+str(spawn[2])+',player,\n' )
            for (x,h,y) in blocks:
                file.write( str(x)+','+str(h)+','+str(y)+','+blocks[x,h,y]+',\n' )

    blocks, spawn, mins, maxs = make_blocks( Z )
    save_blocks( blocks, spawn, mins, maxs )

We then can use Qiskit to create a statevector. For example, here is one with six qubits and a very smooth distrubition.

In [2]:
from qiskit import *
import numpy as np

# create and run some quantum circuit
num = 6
qc = QuantumCircuit(num,num)
qc.h(qc.qregs[0])
statevector = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()

Now we put the statevector into our function to create an island of the size specified by `size` (length x width x height).

In [3]:
ket2csv(statevector,size=[75,75,20])

To see this terrain, use the `/ltbv` chat command with the 'csv2terrain' Minetest mod.

Try different circuits to find out which ones result in nice islands!